In [ ]:
from __future__ import division
from chempy.chemistry import Solute, Equilibrium
from chempy.equilibria import EqSystemLog, composition_balance, EqSystemLin
import periodictable
import numpy as np
import sympy as sp
sp.init_printing()
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
substances = Hp, OHm, NH4p, NH3, H2O = [
    Solute(n, formula=periodictable.formula(n)) for n in [
        'H{+}', 'HO{-}', 'NH3 + H{+}', 'NH3', 'H2O']]
#substances = substances[::-1]
assert (Hp.charge, OHm.charge, NH4p.charge, NH3.charge, H2O.charge) == (1, -1, 1, 0, 0)
init_conc = {Hp: 1e-7, OHm: 1e-7, NH4p: 1e-7, NH3: 1.0, H2O: 55.5}
x0 = [init_conc[k] for k in substances]
H2O_c = init_conc[H2O]
w_autop = Equilibrium({H2O: 1}, {Hp: 1, OHm: 1}, 10**-14/H2O_c)
NH4p_pr = Equilibrium({NH4p: 1}, {Hp: 1, NH3: 1}, 10**-9.26)
equilibria = w_autop, NH4p_pr
[(k, init_conc[k]) for k in substances]

In [ ]:
rs = EqSystemLog(equilibria, substances)
x, sol = rs.root(init_conc)
x

In [ ]:
linsys = EqSystemLin(equilibria, substances)
linx, linsol = linsys.root(init_conc, x0=x)
linsol

In [ ]:
ny = len(substances)
y = sp.symarray('y', ny)
i = sp.symarray('i', ny)
K = Kw, Ka = sp.symbols('K_w K_a')
w_autop.params = Kw
NH4p_pr.params = Ka
ss = sp.symarray('s', ny)
ms = sp.symarray('m', ny)

In [ ]:
f = rs.f(y, list(i)+list(K), ln=sp.log, exp=sp.exp)
f

In [ ]:
linsys.f(y, i, ln=sp.log, exp=sp.exp)

In [ ]:
A, ks = linsys.stoichs_constants(False, ln=sp.log, exp=sp.exp)
from chempy.equilibria import prodpow
list(prodpow(y, A))

In [ ]:
f

In [ ]:
from pyneqsys import SymbolicSys
subs = list(zip(i, x0)) + [(Kw, 10**-14), (Ka, 10**-9.26)]
numf = [_.subs(subs) for _ in f]
neqs = SymbolicSys(list(y), numf)
neqs.solve_scipy([0, 0, 0, 0, 0])

In [ ]:
j = sp.Matrix(1, len(f), lambda _, q: f[q]).jacobian(y)
init_conc_j = {Hp: 1e-10, OHm: 1e-7, NH4p: 1e-7, NH3: 1.0, H2O: 55.5}
xj = rs.as_per_substance_array(init_conc_j)
jarr = np.array(j.subs(zip(y, xj)).subs({Kw: 1e-14, Ka: 10**-9.26}).subs(
            zip(i, xj)))
jarr = np.asarray(jarr, dtype=np.float64)
np.log10(np.linalg.cond(jarr))

In [ ]:
j.simplify()
j

In [ ]:
[s.name for s in rs.substances]

In [ ]:
rs.composition_balance_vectors()

In [ ]:
rs.f(y, list(i)+list(K), True, True, ln=sp.log, exp=sp.exp)

In [ ]:
np.set_printoptions(4, linewidth=120)
scaling = 1e8
for rxn in rs.rxns:
    rxn.params = rxn.params.subs({Kw: 1e-14, Ka: 10**-9.26})

In [ ]:
x, res = rs.root(x0, rref_equil=True, rref_preserv=True)
res.success

In [ ]:
x, res = rs.root({Hp: 1e-11, OHm: 1e-3, NH4p: 1e-3, NH3: 1.0, H2O: 55.5})
res.success

In [ ]:
x, res = rs.root({Hp: 1.7e-11, OHm: 3e-2, NH4p: 3e-2, NH3: 0.97, H2O: 55.5})
res.success

In [ ]:
init_conc

In [ ]:
nc=30
fig = plt.figure(figsize=(16, 6))
ax1 = plt.subplot(2, 2, 1, xscale='log', yscale='log')
Cout_1, ic1, success1 = rs.solve_and_plot(init_conc, Hp, np.logspace(-4, 0, nc), roots_kwargs=dict(rref_equil=False, rref_preserv=False), ax=ax1)
ax2 = plt.subplot(2, 2, 2, xscale='log', yscale='log')
Cout_2, ic2, success2 = rs.solve_and_plot(init_conc, Hp, np.logspace(-4, 0, nc), roots_kwargs=dict(rref_equil=False, rref_preserv=True), ax=ax2)
ax3 = plt.subplot(2, 2, 3, xscale='log', yscale='log')
Cout_3, ic3, success3 = rs.solve_and_plot(init_conc, Hp, np.logspace(-4, 0, nc), roots_kwargs=dict(rref_equil=True, rref_preserv=False), ax=ax3)
ax4 = plt.subplot(2, 2, 4, xscale='log', yscale='log')
Cout_4, ic4, success4 = rs.solve_and_plot(init_conc, Hp, np.logspace(-4, 0, nc), roots_kwargs=dict(rref_equil=True, rref_preserv=True), ax=ax4)
all(success1), all(success2), all(success3), all(success4)

In [ ]:
rs.plot_errors(Cout_4, ic4, Hp)

In [ ]:
import random
subst = tuple(rs.substances)
plt.figure(figsize=(16,6))
for i in range(1, 2*3+1):
    ax = plt.subplot(2, 3, i, xscale='log', yscale='log')
    Cout, ic, success = rs.solve_and_plot(init_conc, Hp, np.logspace(-4, 0, 30), ax=ax)
                                 #norm=True, pres_norm=True, presw=1000)
    plt.title(', '.join([str(s) for s in rs.substances]))
    random.shuffle(rs.substances)
rs.substances = list(subst)
init_conc